In [6]:
import requests

url = 'http://apis.data.go.kr/B552657/AEDInfoInqireService/getAedFullDown'
api_key = 'csNyrGeiYsO1Dw36uAgMBU56sZAkPedxMNwB9u1bqYgGaITRh%2Bsn6MAWl3QfCBCQBqZEgGGLsJKeRr%2FN8l7GkQ%3D%3D'
qs = 'serviceKey={0}&pageNo={1}&numOfRows={2}'.format(api_key, 1, 1000)
response = requests.get(url + "?" + qs)

In [3]:
# response.content

In [7]:
import xmltodict, json
response_dict = xmltodict.parse(response.content)

In [6]:
# response_dict

In [8]:
response_json = json.dumps(response_dict, ensure_ascii=False)

In [10]:
# response_json

In [11]:
response_dict2 = json.loads(response_json)

In [19]:
# type(response_dict2)

In [12]:
response_dict2['response'].keys()
values = response_dict2['response']['body']['items']['item']

In [13]:
# type(values)
# len(values)
values[9]

{'buildAddress': '대전광역시 서구 정림서로 162-18',
 'buildPlace': '관리동 1층',
 'clerkTel': '042-583-1188',
 'manager': '장극순',
 'managerTel': '010-***-****',
 'mfg': '(주)씨유메디칼시스템',
 'model': 'i-padNF1200',
 'org': '강변들보람아파트',
 'rnum': '10',
 'wgs84Lat': '36.3005599077',
 'wgs84Lon': '127.3645882629',
 'zipcode1': '353',
 'zipcode2': '98'}

In [14]:
columns = [
    'num', 'Address', 'DetailedAdress'
]

keys = [
    'rnum', 'buildAddress', 'buildPlace'
]

key_column_map = { key: column for key, column in zip(keys, columns) }
print( key_column_map )
column_key_map = { column: key for key, column in zip(keys, columns) }
print( column_key_map )

len(keys), len(columns)

{'rnum': 'num', 'buildAddress': 'Address', 'buildPlace': 'DetailedAdress'}
{'num': 'rnum', 'Address': 'buildAddress', 'DetailedAdress': 'buildPlace'}


(3, 3)

In [15]:
import pymysql

connection_info = { 'host': 'localhost', 'user': 'root', 'password': 'PASSWORD', 'db': 'demodb', 'charset': 'utf8' }

conn = pymysql.connect(**connection_info)
cursor = conn.cursor()

for value in values:

    sql = "SELECT IFNULL(COUNT(*), 0) FROM aedlocation WHERE num = %s and Address = %s and DetailedAddress = %s"
    cursor.execute(sql, (value['rnum'], value['buildAddress'], value['buildPlace']))
    if cursor.fetchone()[0] > 0:
        continue    

    sql = 'INSERT INTO aedlocation VALUES(%s, %s, %s)'
    cursor.execute(sql, [ value[column_key_map[column]] for column in columns[0:] ])

conn.commit()

conn.close()

OperationalError: (1045, "Access denied for user 'root'@'localhost' (using password: YES)")